Idea:https://www.kaggle.com/code/stefancomanita/regression-with-neural-networks-using-pytorc

# Mass Prediction

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Check GPU support

In [2]:
if torch.cuda.is_available():
    print("CUDA (GPU support) is available in PyTorch!")
    device = torch.device("cuda")
else:
    print("CUDA (GPU support) is not available. Using CPU.")
    device = torch.device("cpu")

CUDA (GPU support) is available in PyTorch!


### Load dataset

In [3]:
HOME = os.getcwd()
csv_path = os.path.join(HOME, 'output.csv')
data = pd.read_csv(csv_path)

In [4]:
data.head()

,image_name,object_id,x_center,y_center,width,height,image_area,area,mass
0,20231103_115922,3.0,0.507812,0.083984,0.845703,0.500000,33923,5454.485113,NaN
1,20231103_122808,0.0,0.099609,0.220703,0.457031,0.546875,32178,6825.504779,50.0
2,20231103_123054,3.0,0.511719,0.089844,0.800781,0.435547,25249,4428.619201,64.0
3,20231103_123054,0.0,0.130859,0.238281,0.529297,0.576172,37373,6555.142200,50.0
4,20231103_123246,0.0,0.132812,0.244141,0.523438,0.583984,36593,6401.413738,50.0


### Drop row without mass and area

In [5]:
data.dropna(subset=['area'], inplace=True)
data.dropna(subset=['mass'], inplace=True)

### Calculate ID counts

In [6]:
value_counts = data['object_id'].value_counts()
value_counts

object_id
0.0     401
3.0     363
7.0     217
16.0    115
19.0     99
6.0      90
5.0      65
20.0     58
8.0      46
12.0     45
21.0     37
1.0      24
18.0     21
2.0       4
11.0      1
Name: count, dtype: int64

In [7]:
keep_table = value_counts >= 30
#keep_table[3.0] = True
print(keep_table)

object_id
0.0      True
3.0      True
7.0      True
16.0     True
19.0    False
6.0     False
5.0     False
20.0    False
8.0     False
12.0    False
21.0    False
1.0     False
18.0    False
2.0     False
11.0    False
Name: count, dtype: bool


### Drop row less than 30

In [8]:
ids_to_keep = value_counts[keep_table].index
data = data[data['object_id'].isin(ids_to_keep)]

### Drop unnecessary column

In [9]:
X = data.drop(columns=['mass', 'image_name', 'x_center', 'y_center', 'width', 'height'], axis=1)
y = data['mass'].copy()

X, y

(      object_id  image_area         area
 1           0.0       32178  6825.504779
 2           3.0       25249  4428.619201
 3           0.0       37373  6555.142200
 4           0.0       36593  6401.413738
 5           3.0       24695  4320.031489
 ...         ...         ...          ...
 3535       16.0       13590  2503.583367
 3538       16.0       11159  2060.027288
 3539        3.0       29779  5497.405914
 3541       16.0       11327  2112.340912
 3542        3.0       29264  5457.362449
 
 [1096 rows x 3 columns],
 1       50.0
 2       64.0
 3       50.0
 4       50.0
 5       64.0
         ... 
 3535    14.0
 3538    14.0
 3539    40.0
 3541    14.0
 3542    40.0
 Name: mass, Length: 1096, dtype: float64)

In [10]:
X_cat = X['object_id'].copy()
X = X.drop(columns=['object_id'])
print(X)

      image_area         area
1          32178  6825.504779
2          25249  4428.619201
3          37373  6555.142200
4          36593  6401.413738
5          24695  4320.031489
...          ...          ...
3535       13590  2503.583367
3538       11159  2060.027288
3539       29779  5497.405914
3541       11327  2112.340912
3542       29264  5457.362449

[1096 rows x 2 columns]


### Create one-hot encoding

In [11]:
# Import necessary library
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the 'object_id' column
encoder = OneHotEncoder(sparse=False)
X_cat_one_hot = encoder.fit_transform(X_cat.values.reshape(-1, 1))

# Convert to a DataFrame
X_cat_one_hot_df = pd.DataFrame(X_cat_one_hot, columns=encoder.get_feature_names_out(['object_id']))

# Reset index of X and concatenate with the one-hot encoded DataFrame
X.reset_index(drop=True, inplace=True)
X = pd.concat([X, X_cat_one_hot_df], axis=1)

print(X.head())
print(X.shape)

   image_area         area  object_id_0.0  object_id_3.0  object_id_7.0  \
0       32178  6825.504779            1.0            0.0            0.0   
1       25249  4428.619201            0.0            1.0            0.0   
2       37373  6555.142200            1.0            0.0            0.0   
3       36593  6401.413738            1.0            0.0            0.0   
4       24695  4320.031489            0.0            1.0            0.0   

   object_id_16.0  
0             0.0  
1             0.0  
2             0.0  
3             0.0  
4             0.0  
(1096, 6)


/config/miniconda3/envs/calorie/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Creating training & testing sets

In [12]:
# Scale the features
X = StandardScaler().fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = map(
    torch.tensor, (X_train, X_test, y_train, y_test)
)

# Create Tensor datasets
train_ds = TensorDataset(X_train.float(), y_train.float())
test_ds = TensorDataset(X_test.float(), y_test.float())

# Data loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

### Define model

In [13]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 25)
        self.fc4 = nn.Linear(25, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    #this is mandatory
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = MLP().to(device)

### Define loss function and optimizer

In [14]:
# Loss function and optimizer
criterion = nn.MSELoss()
#add L2 Regularization(weight_decay), larger regularize the model more
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training & Evalulation

In [15]:
import numpy as np
# Training loop
num_epochs = 400
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()
      outputs = model(inputs).squeeze()
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

    # Evaluation with mean squared error
    model.eval()
    with torch.no_grad():
        mse = 0
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs).squeeze()
            mse += criterion(outputs, targets).item()

    mse /= len(test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, MSE: {mse:.4f}')
    print(f'RMSE:{mse ** (1/2):.4f}')

print("Training complete")

Epoch 1/400, MSE: 2149.9519
RMSE:46.3676
Epoch 2/400, MSE: 1558.5990
RMSE:39.4791
Epoch 3/400, MSE: 273.9137
RMSE:16.5503
Epoch 4/400, MSE: 124.7939
RMSE:11.1711
Epoch 5/400, MSE: 116.2798
RMSE:10.7833
Epoch 6/400, MSE: 109.0562
RMSE:10.4430
Epoch 7/400, MSE: 104.8765
RMSE:10.2409
Epoch 8/400, MSE: 100.5527
RMSE:10.0276
Epoch 9/400, MSE: 98.5555
RMSE:9.9275
Epoch 10/400, MSE: 95.3016
RMSE:9.7623
Epoch 11/400, MSE: 94.3323
RMSE:9.7125
Epoch 12/400, MSE: 92.3900
RMSE:9.6120
Epoch 13/400, MSE: 92.1604
RMSE:9.6000
Epoch 14/400, MSE: 86.3562
RMSE:9.2928
Epoch 15/400, MSE: 83.9446
RMSE:9.1621
Epoch 16/400, MSE: 82.7578
RMSE:9.0971
Epoch 17/400, MSE: 79.5883
RMSE:8.9212
Epoch 18/400, MSE: 79.5538
RMSE:8.9193
Epoch 19/400, MSE: 76.9821
RMSE:8.7739
Epoch 20/400, MSE: 77.9859
RMSE:8.8310
Epoch 21/400, MSE: 75.6951
RMSE:8.7003
Epoch 22/400, MSE: 73.2417
RMSE:8.5581
Epoch 23/400, MSE: 73.2105
RMSE:8.5563
Epoch 24/400, MSE: 72.4131
RMSE:8.5096
Epoch 25/400, MSE: 72.7677
RMSE:8.5304
Epoch 26/400, MS